## LightGBM Hyperparameter Tuning

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
# load our data
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0


In [3]:
# split in to training and testing
from src.data_split import train_test_split
from datetime import datetime

X_train, y_train, X_test, y_test = train_test_split(
    df,
    # have to use datetime here
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(56710, 674)
y_test.shape=(56710,)


#### Define Optuna Objective Function

This guides the search for the optimal hyperparameters. We have to define a function that takes a **trial** object as input and returns a scaler that represents the performance of the model for the given hyperparameters. 

In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-09-19 14:08:27,181] A new study created in memory with name: no-name-cf339535-1bda-405f-86a3-4bac20b2755a


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 164, 'feature_fraction': 0.6114814053482039, 'bagging_fraction': 0.40193248873438386, 'min_child_samples': 84}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 164, 'feature_fraction': 0.6114814053482039, 'bagging_fraction': 0.40193248873438386, 'min_child_samples': 84}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 164, 'feature_fraction': 0.6114814053482039, 'bagging_fraction': 0.40193248873438386, 'min_child_samples': 84}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


[I 2024-09-19 14:09:13,937] Trial 0 finished with value: 2.630036470742395 and parameters: {'num_leaves': 164, 'feature_fraction': 0.6114814053482039, 'bagging_fraction': 0.40193248873438386, 'min_child_samples': 84}. Best is trial 0 with value: 2.630036470742395.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 130, 'feature_fraction': 0.25887394746936, 'bagging_fraction': 0.5991325970936872, 'min_child_samples': 14}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 130, 'feature_fraction': 0.25887394746936, 'bagging_fraction': 0.5991325970936872, 'min_child_samples': 14}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 130, 'feature_fraction': 0.25887394746936, 'bagging_fraction': 0.5991325970936872, 'min_child_samples': 14}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


[I 2024-09-19 14:09:25,038] Trial 1 finished with value: 2.7001423308334744 and parameters: {'num_leaves': 130, 'feature_fraction': 0.25887394746936, 'bagging_fraction': 0.5991325970936872, 'min_child_samples': 14}. Best is trial 0 with value: 2.630036470742395.


...

[I 2024-09-19 14:09:37,634] Trial 2 finished with value: 2.586600027500548 and parameters: {'num_leaves': 67, 'feature_fraction': 0.4693321137290369, 'bagging_fraction': 0.5775197807245391, 'min_child_samples': 59}. Best is trial 2 with value: 2.586600027500548.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 165, 'feature_fraction': 0.3822978841323364, 'bagging_fraction': 0.5342099921117782, 'min_child_samples': 81}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 165, 'feature_fraction': 0.3822978841323364, 'bagging_fraction': 0.5342099921117782, 'min_child_samples': 81}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 165, 'feature_fraction': 0.3822978841323364, 'bagging_fraction': 0.5342099921117782, 'min_child_samples': 81}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


[I 2024-09-19 14:09:55,378] Trial 3 finished with value: 2.6231980469935157 and parameters: {'num_leaves': 165, 'feature_fraction': 0.3822978841323364, 'bagging_fraction': 0.5342099921117782, 'min_child_samples': 81}. Best is trial 2 with value: 2.586600027500548.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


[I 2024-09-19 14:10:14,840] Trial 4 finished with value: 2.5783952270044788 and parameters: {'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}. Best is trial 4 with value: 2.5783952270044788.


The Optuna study object uses the objective function to test the hyperparameters. Since we are trying to optimize mean absolute error, we use **direction = 'minimize'** but this can be changed to direction = 'maximize' if the scoring metric we were trying to optimize was accuracy score or similar metric. 

In [6]:
# best params
best_params = study.best_params
print(f'{best_params=}')

best_params={'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}


In [7]:
# retrain - get_pipeline will add the features and train the lgbm with best params from the study
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Creating pipeline with hyperparameters: {'num_leaves': 94, 'feature_fraction': 0.9524988464955444, 'bagging_fraction': 0.6468377193377184, 'min_child_samples': 52}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001FF887F2F80>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.6468377193377184,
                               feature_fraction=0.9524988464955444,
                               min_child_samples=52, num_leaves=94))])

In [8]:
predictions = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f'{mae=:.4f}')

mae=2.5349


##### Let's Try TimeSeriesSplit

This would be set up differently because order matters. We can see below that the indices increase by one and are not randomly split. This saves us from potentially using future dates in our training data. For example, if we use Kfold split when trying to forecast data we may have the dates mixed together such as February 10th in the training data and February 1st in the testing data. In the real world, we do not have information on future dates so we should not have information on future dates now. 

In [9]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 4)

for train_index, val_index in tss.split(X_train):
    X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
    y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
    print(train_index, val_index)

[   0    1    2 ... 6516 6517 6518] [ 6519  6520  6521 ... 13035 13036 13037]
[    0     1     2 ... 13035 13036 13037] [13038 13039 13040 ... 19554 19555 19556]
[    0     1     2 ... 19554 19555 19556] [19557 19558 19559 ... 26073 26074 26075]
[    0     1     2 ... 26073 26074 26075] [26076 26077 26078 ... 32592 32593 32594]


The X_train and y_train will be split 3 different times. Each time, a model is created and predictions are made. Then, the mean_absolute_error will be added to our scores list. For example, TimeSeriesSplit of X_train and y_train will have 3 mean_absolute_error scores. Then, we are returning the mean of those three scores. 

Following this process we make a study. The study will run 10 different iterations. This study will use different values for our hyperparameters with each iteration. Each iteration will have the TimeSeriesSplit iteration inside of it as well. 

In [10]:
# using time series split
def objective_ts(trial: optuna.trial.Trial) -> float:
    '''Same objective with TimeSeriesSplit.'''
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    # split using kfolds - time stamps not used   
    tss = TimeSeriesSplit(n_splits = 4)
    scores = []

    # set up X_train, X_val, y_train, y_val
    for train_index, val_index in tss.split(X_train):
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        # set up pipeline
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # predictions and evaluation
        predictions = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, predictions)

        # add scores to our empty list
        scores.append(mae)
    
    # return the mean of the scores
    return np.array(scores).mean()


In [11]:
# create a study 
study2 = optuna.create_study(direction= 'minimize')
study2.optimize(objective_ts, n_trials= 5)

[I 2024-09-19 14:10:23,231] A new study created in memory with name: no-name-3fdf0def-9ce3-433b-9ec2-9985d3c5fb39


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 247, 'feature_fraction': 0.7859448581204083, 'bagging_fraction': 0.27863054720887903, 'min_child_samples': 12}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 247, 'feature_fraction': 0.7859448581204083, 'bagging_fraction': 0.27863054720887903, 'min_child_samples': 12}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 247, 'feature_fraction': 0.7859448581204083, 'bagging_fraction': 0.27863054720887903, 'min_child_samples': 12}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 247, 'feature_fraction': 0.7859448

[I 2024-09-19 14:11:00,746] Trial 0 finished with value: 3.241083142838196 and parameters: {'num_leaves': 247, 'feature_fraction': 0.7859448581204083, 'bagging_fraction': 0.27863054720887903, 'min_child_samples': 12}. Best is trial 0 with value: 3.241083142838196.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 74, 'feature_fraction': 0.44430489117541355, 'bagging_fraction': 0.7689868487543543, 'min_child_samples': 63}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 74, 'feature_fraction': 0.44430489117541355, 'bagging_fraction': 0.7689868487543543, 'min_child_samples': 63}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 74, 'feature_fraction': 0.44430489117541355, 'bagging_fraction': 0.7689868487543543, 'min_child_samples': 63}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 74, 'feature_fraction': 0.44430489117

[I 2024-09-19 14:11:14,060] Trial 1 finished with value: 3.2715777359107747 and parameters: {'num_leaves': 74, 'feature_fraction': 0.44430489117541355, 'bagging_fraction': 0.7689868487543543, 'min_child_samples': 63}. Best is trial 0 with value: 3.241083142838196.


...

[I 2024-09-19 14:11:27,104] Trial 2 finished with value: 3.5218787090577406 and parameters: {'num_leaves': 140, 'feature_fraction': 0.22803628320847055, 'bagging_fraction': 0.7645438910872426, 'min_child_samples': 5}. Best is trial 0 with value: 3.241083142838196.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 62, 'feature_fraction': 0.4698291145752985,

[I 2024-09-19 14:11:39,850] Trial 3 finished with value: 3.1889324092902167 and parameters: {'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}. Best is trial 3 with value: 3.1889324092902167.


Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 82, 'feature_fraction': 0.8630112792051319, 'bagging_fraction': 0.9517362734652814, 'min_child_samples': 86}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 82, 'feature_fraction': 0.8630112792051319, 'bagging_fraction': 0.9517362734652814, 'min_child_samples': 86}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 82, 'feature_fraction': 0.8630112792051319, 'bagging_fraction': 0.9517362734652814, 'min_child_samples': 86}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:
Creating pipeline with hyperparameters: {'metric': 'mae', 'verbose': -1, 'num_leaves': 82, 'feature_fraction': 0.86301127920513

[I 2024-09-19 14:11:56,379] Trial 4 finished with value: 3.521914626304531 and parameters: {'num_leaves': 82, 'feature_fraction': 0.8630112792051319, 'bagging_fraction': 0.9517362734652814, 'min_child_samples': 86}. Best is trial 3 with value: 3.1889324092902167.


In [12]:
# best params
best_params2 = study2.best_params
print(f'{best_params2=}')

best_params2={'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}


In [13]:
# retrain
pipeline2 = get_pipeline(**best_params2)
pipeline2.fit(X_train, y_train)

Creating pipeline with hyperparameters: {'num_leaves': 62, 'feature_fraction': 0.4698291145752985, 'bagging_fraction': 0.715706101008309, 'min_child_samples': 54}
Added feature transformer for average rides
Added temporal features engineer
Pipeline created:


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001FF887F2F80>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.715706101008309,
                               feature_fraction=0.4698291145752985,
                               min_child_samples=54, num_leaves=62))])

This model has a higher mean absolute error than the kfold split method but this is okay because with real data it is harder to predict the future and kfold does not have a realistic way of training and testing data because the dates do not matter in the splits. For this reason, the kfold splits have access to future dates and thus a lower mean absolute error is returned.

In [14]:
preds = pipeline2.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f'{mae=:.4f}')

mae=2.6031
